In [16]:
import requests
from pprint import pprint
import json 

In [34]:
data_to_send = {
    "config":
        {
        "QAOA":{
            "general":{ # General parameters in the QAOA algorithm
                "func_find_par": { # Name of the function in the QAOA algorithm
                    "method": "COBYLA", # Method to use in optimization
                    "tolerance": 0.001, # Tolerance of the optimization
                    "options": {"maxiter": 100000} # Options of the optimization
                },
                "func_find_best_parameters": {
                    "pool_size": 10, # Number of virtual processors to use
                    "iter_init": 100, # Number of solutions to generate for every P-value
                    "best_number": 15, # Number of best solutions to keep for the next P-value
                    "p_start": 3, # Starting P-value
                    "p_end": 10, # Ending P-value
                    "use_next_params": True, # Use the parameters learned in the previous P-value
                    "beta_corr_thr": 0.9, # Threshold for the beta correction to be accepted as good solution
                    "gamma_corr_thr": 0.9 # Threshold for the gamma correction to be accepted as good solution
                }
            },
            "jsp": {
                "max_time": 4, # Maximum time that the solution can take
                "problem_version": "optimization" # Version of the problem to solve (optimization or decizion)
            },
            "maxcut": {
                "n": -1, # Number of vertices in random graph (if -1, data taken from input) 
                #"a": 1
            }
        },
        
        "D-Wave": {
            "jsp":{
                "mode": "sim_pyqubo",
                "num_reads": 1000,
                "weight_one_hot": 3,
                "weight_precedence": 1,
                "weight_share":2
            }
        }
    }
}


In [35]:
data = {
    "ExactCover": {
        "routes": "[{1,4,7},{1,4},{4,5,7},{3,5,6},{2,3,6,7},{2,7}]"
    },
    "MaxCut": {
        "V": "[0,4,5,8,1,3,7,9,6,2]",
        "E": "[(0,4),(0,5),(0,8),(4,6),(4,7),(5,8),(5,1),(8,1),(1,9),(3,7),(3,2),(3,9),(7,6),(9,2),(6,2)]"
    },
    "MaxCut_Weighted": {
        "V": "[0,1,2,3,4]",
        "E": "[(2,4),(4,1),(1,0),(0,2),(0,3),(3,1)]",
        "W": "[4,9,5,5,6,2]"
    },
    "JSP": {
        "Tasks": "{'cupcakes':[('mixer',2) ,('oven',1)],'smoothie':[('mixer',1)],'lasagna':[('oven',2)]}"
    },
}

In [36]:
Problems_to_solve = {
    "ExactCover": 0,
    "MaxCut": 1,
    "MaxCut_Weighted": 1,
    "JSP": 0
}

In [40]:
data_to_send["solve"] = {
    "input_data": [],
    "problems": [],}
for problem in Problems_to_solve:
    if Problems_to_solve[problem] == 1:
        data_to_send["solve"]["input_data"].append(data[problem])
        data_to_send["solve"]["problems"] += [problem]

print(requests.post("http://127.0.0.1:8000/solve/", json=json.dumps(data_to_send)).json())

<Response [200]>


In [ ]:
responses_IN_PROGRESS = []
REST_API_Solve = "http://kwanty.com/solve/"
for problem in Problems_to_solve:
    if Problems_to_solve[problem] == 1:
        responses_IN_PROGRESS.append(requests.post(REST_API_Solve+problem, json=config))
        print(f"Solving problem: {problem}, at {REST_API_Solve+problem}")

Solving problem: MaxCut, at http://kwanty.com/solve/MaxCut


In [ ]:
# Checking the status
test_response_Solving = [{"id": 1, "problem": "MaxCut", "status": "Solving", "solution": "0101100110"}]
test_response_Solved = [{"id": 1, "problem": "MaxCut", "status": "Solved", "solution": "0101100110"}]


response_DONE = []
REST_API_Results = "http://kwanty.com/results/"
# for response in responses_IN_PROGRESS:
for response in test_response_Solved:
    print(f"Checking status of problem: {response['problem']}, at {REST_API_Results+str(response['id'])}")
    # response_from_server = requests.get(REST_API_Results+str(response["id"]))
    response_from_server = test_response_Solved[0]
    if response_from_server["status"] == "Solved":
        response_DONE.append(response_from_server)
        print(f"Problem {response['problem']} with id: {response['id']} - solved")





Checking status of problem: MaxCut, at http://kwanty.com/results/1
Problem MaxCut with id: 1 - solved
